In [1]:
import numpy as np
import pandas as pd
import tweepy
from textblob import TextBlob
from tweepy import OAuthHandler

In [2]:
consumer_key ='68Ud1dKuzuikwF8wqKYlBdork' 
consumer_secret = 'iSOPIPn7TVbwypbt9cA8MprnFIj0BI0jwpHGOSkwcTd53tWjIH'

access_token = '363234238-g2t4Pu0WjeNzVNeQnJsVKHGEh7wIQVt8k390r08J'
access_token_secret = '2Tc8FlOxzAY45j91vCcwoM9FnKOZhyOMs74PT5vRSU8px'

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)

In [3]:
def senti_kc(name):
    public_tweets = api.search(name)
    tweet_twt = []
    tweet_pol = []
    tweet_sen = []
    for tweet in public_tweets:
        tweet_twt.append(tweet.text)
        #print(tweet.text)
        analysis = TextBlob(tweet.text)
        tweet_pol.append(analysis.sentiment.polarity)
        tweet_sen.append(analysis.sentiment.subjectivity)
        #print(analysis.sentiment)
        #print("")

    df = pd.DataFrame(np.column_stack([tweet_twt, tweet_pol, tweet_sen]), columns=['Tweet_desc', 'Sentiment_Polarity', 'Sentiment_Subjectivity'])
    return df

In [15]:
trump = senti_kc('@realDonaldTrump')
modi  = senti_kc('@narendramodi')

In [16]:
trump.to_csv('trump.csv', encoding='utf-8')
modi.to_csv('modi.csv', encoding='utf-8')

In [17]:
import time
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
from textblob import TextBlob
import matplotlib.pyplot as plt
import re

"# -- coding: utf-8 --"

def calctime(a):
    return time.time()-a

positive=0
negative=0
compound=0

count=0
initime=time.time()
plt.ion()

class listener(StreamListener):
    
    def on_data(self,data):
        global initime
        t=int(calctime(initime))
        all_data=json.loads(data)
        tweet=all_data["text"].encode("utf-8")
        #username=all_data["user"]["screen_name"]
        tweet=" ".join(re.findall("[a-zA-Z]+", tweet))
        blob=TextBlob(tweet.strip())

        global positive
        global negative     
        global compound  
        global count
        
        count=count+1
        senti=0
        for sen in blob.sentences:
            senti=senti+sen.sentiment.polarity
            if sen.sentiment.polarity >= 0:
                positive=positive+sen.sentiment.polarity   
            else:
                negative=negative+sen.sentiment.polarity  
        compound=compound+senti        
        print (count)
        print (tweet.strip())
        print (senti)
        print (t)
        print (str(positive) + ' ' + str(negative) + ' ' + str(compound)) 
        
    
        plt.axis([ 0, 70, -20,20])
        plt.xlabel('Time')
        plt.ylabel('Sentiment')
        plt.plot([t],[positive],'go',[t] ,[negative],'ro',[t],[compound],'bo')
        plt.show()
        plt.pause(0.0001)
        if count==200:
            return False
        else:
            return True
        
    def on_error(self,status):
        print (status)


In [24]:
import re        
 
consumer_key ='68Ud1dKuzuikwF8wqKYlBdork' 
consumer_secret = 'iSOPIPn7TVbwypbt9cA8MprnFIj0BI0jwpHGOSkwcTd53tWjIH'

access_token = '363234238-g2t4Pu0WjeNzVNeQnJsVKHGEh7wIQVt8k390r08J'
access_token_secret = '2Tc8FlOxzAY45j91vCcwoM9FnKOZhyOMs74PT5vRSU8px'

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

#api = tweepy.API(auth)

twitterStream=  Stream(auth, listener(count))

twitterStream.filter(track=['Donald Trump'])


TypeError: cannot use a string pattern on a bytes-like object